In [ ]:

# --- Data load (works in Voici/JupyterLite) ---
from pathlib import Path
import numpy as np, pandas as pd

HERE = Path(".").resolve()
DATA = HERE / "data"          # put meta.csv, tuning_curves.npz, psth.npz here
PKG = str(DATA)               # backward-compat if older code used PKG

def must_exist(p: Path):
    if not p.exists():
        raise FileNotFoundError(f"Required file not found: {p}")
    return p

try:
    meta = pd.read_csv(must_exist(DATA / "meta.csv"))
    tun  = np.load(must_exist(DATA / "tuning_curves.npz"))
    psth = np.load(must_exist(DATA / "psth.npz"))
    angles = tun["angles"]; tun_curves = tun["curves"]
    time   = psth["t"];     psth_curves = psth["curves"]
except Exception:
    # Fallback demo data (so the page still works if real files are missing)
    angles = np.linspace(0, 350, 36)
    time   = np.linspace(-0.5, 1.0, 151)
    rng = np.random.default_rng(0)
    meta = pd.DataFrame({
        "unit_id": np.arange(1, 31),
        "layer": rng.choice(["SG","G","IG"], size=30),
        "OSI": np.round(rng.uniform(0.1, 0.95, size=30), 3),
        "selectivity": np.round(rng.uniform(0.4, 1.1, size=30), 3),
    })
    # demo tuning curves
    tc = []
    for i in range(30):
        peak = rng.choice(angles)
        curve = 5 + 15*np.exp(-0.5*((angles-peak+360)%360/30)**2) + rng.normal(0,0.5,size=len(angles))
        tc.append(np.clip(curve, 0, None))
    tun_curves = np.stack(tc, axis=0)
    # demo psth
    pc = []
    for i in range(30):
        mu = rng.uniform(0.2, 0.7)
        curve = 6 + 20*np.exp(-0.5*((time-mu)/0.12)**2) + rng.normal(0,0.8,size=len(time))
        pc.append(np.clip(curve, 0, None))
    psth_curves = np.stack(pc, axis=0)


In [ ]:

# --- Widgets & layout (left controls + right plot) ---
import ipywidgets as W
from ipywidgets import AppLayout, Layout
from IPython.display import display

layers = sorted(meta["layer"].dropna().unique().tolist())
osi_min, osi_max  = float(meta["OSI"].min()), float(meta["OSI"].max())
sel_min, sel_max  = float(meta["selectivity"].min()), float(meta["selectivity"].max())

layer_w = W.SelectMultiple(options=layers, value=tuple(layers), description="Layer",
                           layout=Layout(width="260px", height="120px"))
osi_w   = W.FloatRangeSlider(description="OSI range", min=0, max=1, step=0.01,
                             value=[max(0, osi_min), min(1, osi_max)], readout_format=".2f",
                             layout=Layout(width="260px"))
sel_w   = W.FloatRangeSlider(description="Sel range", min=0, max=1.2, step=0.01,
                             value=[max(0, sel_min), min(1.2, sel_max)], readout_format=".2f",
                             layout=Layout(width="260px"))
plot_w  = W.ToggleButtons(options=["Tuning","PSTH"], value="Tuning", description="Plot",
                          layout=Layout(width="260px"))
idx_w   = W.IntSlider(description="Unit index", min=0, max=len(meta)-1, step=1, value=0,
                      continuous_update=False, layout=Layout(width="260px"))

controls = W.VBox([layer_w, osi_w, sel_w, plot_w, idx_w],
                  layout=Layout(width="280px", min_width="280px", flex="0 0 280px"))

plot_out   = W.Output(layout=Layout(width="100%", min_height="640px", overflow="auto",
                                    border="1px solid #eee", padding="6px"))
status_out = W.HTML(layout=Layout(margin="6px 0 0 0"))

app = AppLayout(
    header=None, left_sidebar=controls, center=plot_out, right_sidebar=None, footer=status_out,
    pane_widths=["300px", "1fr", 0], pane_heights=[0, "auto", "40px"]
)
display(app)


In [ ]:

# --- Render logic ---
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as W
from IPython.display import display

def filtered_indices():
    df = meta.copy()
    if layer_w.value:
        df = df[df["layer"].isin(list(layer_w.value))]
    lo, hi = osi_w.value
    df = df[(df["OSI"] >= lo) & (df["OSI"] <= hi)]
    lo2, hi2 = sel_w.value
    df = df[(df["selectivity"] >= lo2) & (df["selectivity"] <= hi2)]
    return df.index.to_numpy(), df

def newfig():
    plt.close("all")
    fig = plt.figure(figsize=(8.5,5.0), dpi=110)
    plt.grid(True, alpha=0.25)
    return fig

def render(*_):
    idxs, df = filtered_indices()
    n = len(df)
    with plot_out:
        plot_out.clear_output(wait=True)
        if n == 0:
            display(W.HTML("<b>No units match the filters.</b>"))
            status_out.value = "No matching units."
            return
        pos = min(idx_w.value, n-1)
        unit_global_idx = int(idxs[pos])
        row = meta.iloc[unit_global_idx]
        newfig()
        if plot_w.value == "Tuning":
            plt.plot(angles, tun_curves[unit_global_idx], marker="o")
            plt.title(f"Tuning — unit {int(row['unit_id'])} | layer={row['layer']} | OSI={row['OSI']:.2f}")
            plt.xlabel("Orientation (deg)"); plt.ylabel("Response (a.u.)")
        else:
            plt.plot(time, psth_curves[unit_global_idx])
            plt.title(f"PSTH — unit {int(row['unit_id'])} | layer={row['layer']} | OSI={row['OSI']:.2f}")
            plt.xlabel("Time (s)"); plt.ylabel("Rate (Hz)")
        plt.tight_layout(); plt.show()
    status_out.value = f"<span>{n} units match filters. Showing {pos+1}/{n} (global idx {unit_global_idx}).</span>"

for w in (layer_w, osi_w, sel_w, plot_w, idx_w):
    w.observe(render, names="value")
render()
